In [2]:
import pybel
import openbabel
import numpy as np

In [21]:
mol = pybel.readstring("smi", "CCCC")
mol.make3D()
for atom in mol:
    print(atom)

Atom: 6 (0.90 -0.04 0.04)
Atom: 6 (2.42 -0.03 0.06)
Atom: 6 (2.97 0.70 1.28)
Atom: 6 (4.49 0.70 1.29)
Atom: 1 (0.53 -0.57 -0.84)
Atom: 1 (0.51 0.98 0.02)
Atom: 1 (0.51 -0.55 0.93)
Atom: 1 (2.79 0.45 -0.86)
Atom: 1 (2.79 -1.07 0.04)
Atom: 1 (2.60 0.21 2.20)
Atom: 1 (2.60 1.73 1.29)
Atom: 1 (4.88 -0.32 1.31)
Atom: 1 (4.86 1.23 2.18)
Atom: 1 (4.88 1.21 0.41)


In [15]:
x = mol.OBMol.GetAtom(1).x()

In [16]:
print(x)

0.9003783942423311


In [5]:
forcefields = pybel.forcefields

In [6]:
forcefields

['gaff', 'ghemical', 'mmff94', 'mmff94s', 'uff']

In [17]:
ff_instance = pybel.ob.OBForceField.FindForceField("mmff94")

In [18]:
ff_instance.Setup(mol.OBMol)

True

In [19]:
e = ff_instance.Energy(False)

In [20]:
e # in kcal/mol

-5.074118008751814

In [ ]:
# jen note:
# windows installation of openbabel in miniconda did not work
# the setup function failed
# the obenergy in the terminal in windows also did not work
# gave several errors for missing components
# same input worked just fine for ubuntu...<sigh>
# only issue with this is that it will require an xyz file to work


# aside
# idea: set new vetee env with my version to test
# API with Kaplan; keep old vetee env with kaplan
# in case transfer to new API goes poorly

In [22]:
# now try the same procedure except first
# the coordinates must be set manually (so we
# can calculate the energy of a given conformer)

In [31]:
num_atoms = 14
mol = pybel.ob.OBMol()
for i in range(num_atoms):
    atom = pybel.ob.OBAtom()
    atom.SetVector(coords[i])
    if i < 4:
        atom.SetAtomicNum(6)
    else:
        atom.SetAtomicNum(1)

TypeError: in method 'OBAtom_SetVector', argument 2 of type 'OpenBabel::vector3 const &'

In [46]:
mol = pybel.readstring("smi", "CCCC")
mol.make3D()
obmol = mol.OBMol
for i, atom in enumerate(mol):
    x,y,z = atom.coords
    print(x,y,z)
    print(atom.coords[0])
    atom.coords[0] = coords[i][0]
    print(atom.coords)

1.00461678470296 -0.03571314528267805 -0.07894247658819051
1.00461678470296


TypeError: 'tuple' object does not support item assignment

In [6]:
mol = pybel.readstring("smi", "CCCC")
mol.make3D()
obmol = mol.OBMol
for i, atom in enumerate(mol):
    obmol.SetVector(coords[i])

AttributeError: 'OBMol' object has no attribute 'SetVector'

In [55]:
# generate an openbabel molecule
obmol = openbabel.OBMol()
# add coordinates for each atom
for i, atom in enumerate(coords):
    obatom = openbabel.OBAtom()
    if i < 4:
        atomicnum = 6
    else:
        atomicnum = 1
    obatom.SetAtomicNum(atomicnum)
    obatom.SetVector(coords[i][0], coords[i][1], coords[i][2])
    obmol.AddAtom(obatom)
# set charge, multiplicity, and comments (title)
obmol.SetTotalCharge(0)
obmol.SetTotalSpinMultiplicity(1)
obmol.SetTitle("test butane")

In [52]:
ff_instance = pybel.ob.OBForceField.FindForceField("mmff94")

In [1]:
ff_instance.Setup(obmol)

NameError: name 'ff_instance' is not defined

In [56]:
obmol.AddHydrogens()

True

In [2]:
# note:
# combine obmol setup and readstring
# to create a molecule with new coordinates
# hopefully this will work

In [10]:
pybel_mol = pybel.readstring("smi", "CCCC")
pybel_mol.make3D()
obabel_mol = pybel_mol.OBMol

In [40]:
for obatom in openbabel.OBMolAtomIter(obabel_mol):
    print(obatom.GetAtomicNum(), obatom.GetAtomicMass(),
          obatom.GetIdx())

6 12.0107 1
6 12.0107 2
6 12.0107 3
6 12.0107 4
1 1.00794 5
1 1.00794 6
1 1.00794 7
1 1.00794 8
1 1.00794 9
1 1.00794 10
1 1.00794 11
1 1.00794 12
1 1.00794 13
1 1.00794 14


In [ ]:
# jen notes
# the bond angles and torsions are given in indices
# not clear whether these indices correspond
# to the idx for the atom (idx seems to go from 1-number
# of atoms)
# looking at atomic connections, the idx is +1 from index

In [ ]:
obmol = openbabel.OBMol()
for atom in range(14):
    atom = openbabel.OBAtom()
    

In [43]:
just_carbons = pybel.readstring("smi", "CCCC")

In [44]:
for t in openbabel.OBMolTorsionIter(just_carbons.OBMol):
    print(t)
# interestingly, only one torsion angle for n-butane
# when hydrogens are ignored

(0, 1, 2, 3)


In [45]:
# test with benzene
# using kekule form
benzene = pybel.readstring("smi", "C1=CC=CC=C1")

In [46]:
for t in openbabel.OBMolTorsionIter(benzene.OBMol):
    print(t)

(5, 0, 1, 2)
(0, 1, 2, 3)
(1, 2, 3, 4)
(2, 3, 4, 5)
(3, 4, 5, 0)
(1, 0, 5, 4)


In [55]:
# see if it detected the ring
for r in openbabel.OBMolRingIter(benzene.OBMol):
    print(r) # obring object
    print(r.Size()) # 6, as expected
    print(r.IsAromatic()) # True
    # somehow knows benzene is aromatic! nice
    print(r.GetType()) # benzene
    print(r.PathSize()) # 6
    for atom in openbabel.OBMolAtomIter(benzene.OBMol):
        print(r.IsInRing(atom.GetIdx()))
    
    

<openbabel.OBRing; proxy of <Swig Object of type 'OpenBabel::OBRing *' at 0x7fc5eac9ac60> >
6
True
benzene
6
True
True
True
True
True
True


In [56]:
# now try to reproduce a molecule
orig_mol = pybel.readstring("smi", "CCCC").OBMol
# add this stage you can find the backbone (since
# mol does not have hydrogens) and rings
# essentially determine which dihedral angles
# are important

In [63]:
orig_mol_wH = pybel.readstring("smi", "CCCC")
orig_mol_wH.make3D()
orig_mol_wH = orig_mol_wH.OBMol

In [70]:
# details for OBMol::BeginModify()
# Call when making many modifications – clears conformer/rotomer
# data. The method "turns off" perception routines, improving
# performance. Changes in molecular structure will be
# re-considered after modifications.

# add atoms
for i, atom in enumerate(openbabel.OBMolAtomIter(orig_mol_wH)):
    newatom = openbabel.OBAtom()
    newatom.SetAtomicNum(atom.GetAtomicNum())
    # needs to be explicit (doesn't accept tuple, list, np.array)
    newatom.SetVector(coords[i][0], coords[i][1], coords[i][2])

In [23]:
def get_bonds_list(obmol):
    """Get a list of bonds from an openbabel molecule.
    
    Parameters
    ----------
    obmol : openbabel.OBMol object
    
    Returns
    -------
    bonds_list : list()
        Each element is a tuple of 3 integers:
        atom1 index, atom2 index, and bond order.
    
    """
    # this is how we iterate over the bonds in the
    # molecule and find out which atoms are connected
    # to one another
    bonds_list = []
    for obbond in openbabel.OBMolBondIter(obmol):
        #bond_len = obbond.GetLength()
        atom1 = obbond.GetBeginAtom().GetIdx()-1
        atom2 = obbond.GetEndAtom().GetIdx()-1
        bond_order = obbond.GetBondOrder()
        bonds_list.append((atom1, atom2, bond_order))
    return bonds_list

In [24]:
def get_angles_list(obmol):
    """Get a list of angles from an openbabel molecule.
    
    Parameters
    ----------
    obmol : openbabel.OBMol object
    
    Returns
    -------
    angles_list : list()
        Each element in the list is a tuple of 3 integers,
        each integer representing an atom index. The first
        index is the bond vertex. Example: [(a,b,c)] means
        c - a - b is the angle.
    
    """
    angles_list = []
    for obangle in openbabel.OBMolAngleIter(obmol):
        angles_list.append(obangle) 
        # note obangle is just a tuple not an ob object
    return angles_list

In [25]:
def get_torsion_list(obmol):
    """Get a list of torsion angles from an openbabel molecule.
    
    Parameters
    ----------
    obmol : openbabel.OBMol object
    
    Returns
    -------
    torsion_list : list()
        Each element in the list is a tuple of 4 integers,
        each integer representing an atom index. Example:
        [(a,b,c,d)] means a-b-c-d is the torsion (dihedral)
        angle.
    
    """
    torsion_list = []
    for obtorsion in openbabel.OBMolTorsionIter(obmol):
        torsion_list.append(obtorsion) 
        # again, obtorsion is a tuple object, not ob object
    return torsion_list

In [30]:
def new_from_coords(orig_mol, coords):
    """Generate a new openbabel molecule from an existing mol.
    
    Parameters
    ----------
    orig_mol : openbabel.OBMol object
        The original openbabel molecule to use (for bonds,
        angles, atomic numbers, torsion angles, and rings).
        Should already have hydrogens added and be 3D.
    coords : np.array(shape=(num_atoms, 3), dtype="float")
        The coordinates to create the new molecule with.
    
    Notes
    -----
    If I can figure out how to modify an openbabel molecule
    by changing a dihedral value, then this function will
    become unnecessary.
    
    Returns
    -------
    new_mol : openbabel.OBMol object
        Has the same connections as the orig_mol, except
        with new coordinates.
    
    """
    # new molecule we have coordinates for already
    new_mol = openbabel.OBMol()
    new_mol.BeginModify()
    
    # add atoms
    for i, atom in enumerate(openbabel.OBMolAtomIter(orig_mol)):
        newatom = openbabel.OBAtom()
        newatom.SetAtomicNum(atom.GetAtomicNum())
        # needs to be explicit (doesn't accept tuple, list, np.array)
        newatom.SetVector(coords[i][0], coords[i][1], coords[i][2])
        # add atom instance to mol instance (returns True if successful)
        result = new_mol.AddAtom(newatom)
        if not result:
            print(newatom.GetVector(), newatom.Idx(), newatom.GetAtomicNum())
            raise ValueError("Unable to add atom to molecule")
    # add bonds
    for atom1, atom2, BO in get_bonds_list(orig_mol):
        # needs +1 since AddBond accepts Idx as input
        result = new_mol.AddBond(atom1+1, atom2+1, BO)
        if not result:
            print(atom1+1, atom2+1, BO)
            raise ValueError("Unable to add bond to molecule")
    
    # done creating molecule
    new_mol.EndModify()
    return new_mol

In [31]:
"""
Atom: 6 (0.90 -0.04 0.04)
Atom: 6 (2.42 -0.03 0.06)
Atom: 6 (2.97 0.70 1.28)
Atom: 6 (4.49 0.70 1.29)
Atom: 1 (0.53 -0.57 -0.84)
Atom: 1 (0.51 0.98 0.02)
Atom: 1 (0.51 -0.55 0.93)
Atom: 1 (2.79 0.45 -0.86)
Atom: 1 (2.79 -1.07 0.04)
Atom: 1 (2.60 0.21 2.20)
Atom: 1 (2.60 1.73 1.29)
Atom: 1 (4.88 -0.32 1.31)
Atom: 1 (4.86 1.23 2.18)
Atom: 1 (4.88 1.21 0.41)
"""
coords = [
    (0.90, -0.04,  0.04),
    (2.42, -0.03,  0.06),
    (2.97,  0.70,  1.28),
    (4.49,  0.70,  1.29),
    (0.53, -0.57, -0.84),
    (0.51,  0.98,  0.02),
    (0.51, -0.55,  0.93),
    (2.79,  0.45, -0.86),
    (2.79, -1.07,  0.04),
    (2.60,  0.21,  2.20),
    (2.60,  1.73,  1.29),
    (4.88, -0.32,  1.31),
    (4.86,  1.23,  2.18),
    (4.88,  1.21,  0.41),
]

In [32]:
# lol there is an OBMol method called SetTorsion....
# so it may be soooo much easier to use OB for my code

In [41]:
test1 = pybel.readstring("smi", "CCCC")
test1.make3D()
test1a = test1.OBMol

result = new_from_coords(test1a, coords)
print(result)
for atom in openbabel.OBMolAtomIter(result):
    print(atom.x(), atom.y(), atom.z())
    

<openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x7f481ca192d0> >
0.9 -0.04 0.04
2.42 -0.03 0.06
2.97 0.7 1.28
4.49 0.7 1.29
0.53 -0.57 -0.84
0.51 0.98 0.02
0.51 -0.55 0.93
2.79 0.45 -0.86
2.79 -1.07 0.04
2.6 0.21 2.2
2.6 1.73 1.29
4.88 -0.32 1.31
4.86 1.23 2.18
4.88 1.21 0.41


In [34]:
o = pybel.readstring("smi", "CCCC")
o.make3D()
o = o.OBMol
result = get_angles_list(o)
print(result)
for bond in result:
    a,b,c = bond
    print(a,b,c)

[(0, 1, 4), (0, 1, 5), (0, 1, 6), (0, 4, 5), (0, 4, 6), (0, 5, 6), (1, 0, 2), (1, 0, 7), (1, 0, 8), (1, 2, 7), (1, 2, 8), (1, 7, 8), (2, 1, 3), (2, 1, 9), (2, 1, 10), (2, 3, 9), (2, 3, 10), (2, 9, 10), (3, 2, 11), (3, 2, 12), (3, 2, 13), (3, 11, 12), (3, 11, 13), (3, 12, 13)]
0 1 4
0 1 5
0 1 6
0 4 5
0 4 6
0 5 6
1 0 2
1 0 7
1 0 8
1 2 7
1 2 8
1 7 8
2 1 3
2 1 9
2 1 10
2 3 9
2 3 10
2 9 10
3 2 11
3 2 12
3 2 13
3 11 12
3 11 13
3 12 13


In [44]:
def get_ff_energy(obmol, ff):
    """Get the forcefield energy of an openbabel molecule.
    
    Parameters
    ----------
    obmol : openbabel.OBMol object
        The molecule for which to calculate energy.
    ff : str
        The forcefield to use.
    
    Raises
    ------
    AssertionError
        The ff is not available.
    ValueError
        The calculation did not work.
    
    Returns
    -------
    energy : float
        The energy of obmol calculated using ff.
        Units should be kcal/mol
    
    """
    forcefields = pybel.forcefields
    assert ff in forcefields
    ff_instance = pybel.ob.OBForceField.FindForceField(ff)
    result = ff_instance.Setup(obmol)
    if not result:
        raise ValueError("Unable to setup forcefield for molecule")
    energy = ff_instance.Energy(False)
    return energy

In [45]:
e = get_ff_energy(result, "mmff94")

In [46]:
print(e)

-4.938610435659327


In [3]:
# now I want to try and create a molecule
# then change its dihedrals
# then see if the coordinates changed
# then calculate the energy
# don't create hydrogens this time
# so we keep only one torsion angle
# actually that won't work; need to call
# make3D() otherwise no coordinates are set
orig_mol = pybel.readstring("smi", "CCCC")
orig_mol.make3D()
orig_mol = orig_mol.OBMol
for atom in openbabel.OBMolAtomIter(orig_mol):
    print(atom.x(), atom.y(), atom.z())
t = orig_mol.GetTorsion(1,2,3,4) # note these have to be Idx not index starting from zero (seg fault)
print(t) # value in degrees

# set torsion requires atom object instances
# and input is in radians not degrees
atom1 = orig_mol.GetAtom(1)
atom2 = orig_mol.GetAtom(2)
atom3 = orig_mol.GetAtom(3)
atom4 = orig_mol.GetAtom(4)

newt = 1 # radian ~57 degrees

orig_mol.SetTorsion(atom1, atom2, atom3, atom4, newt)
for atom in openbabel.OBMolAtomIter(orig_mol):
    print(atom.x(), atom.y(), atom.z())
t = orig_mol.GetTorsion(1,2,3,4) # note these have to be Idx not index starting from zero (seg fault)
print(t) # value in degrees

0.91883598105288 0.06152576143085063 0.02770234091902385
2.438724955724524 0.07115832068566141 0.037337533024885704
2.981880081594062 1.0803123614719654 1.0467740861253274
4.50176907828565 1.0899443604204602 1.0564092614194764
0.5486773378237261 -0.6667174278310838 -0.7007430834070775
0.522908924030731 1.0453544203353582 -0.24321606366849535
0.5229089343255305 -0.20966586097529188 1.0114557467888372
2.804276994593554 0.31478097546258704 -0.9669667478462978
2.8042769873347893 -0.9332140685687141 0.2806791436027195
2.616327991048092 0.8366897163145175 2.0510783515811233
2.6163282852184735 2.084684853312848 0.8034325396701769
4.897695786808177 0.10611486358156216 1.327325158741026
4.871928114934778 1.8181855848124682 1.7848564395462643
4.897696101963415 1.3611382232752307 0.07265646169740268
179.99998444374467
0.91883598105288 0.06152576143085063 0.02770234091902385
2.438724955724524 0.07115832068566141 0.037337533024885704
2.981880081594062 1.0803123614719654 1.0467740861253274
2.4637760

In [4]:
# okay now we need a way to select important torsion angles
# any torsion angles that are completely in rings should
# probably not change
# torsion angles that can result in H-bonding should
# be included, which means we need to screen existing
# hydrogens for their suitability for H-bonding

In [7]:
test1 = pybel.readstring("smi", "CCCCC=C=C(C)CC")
test1a = test1.OBMol

for torsion in openbabel.OBMolTorsionIter(test1a):
    print(torsion)

(0, 1, 2, 3)
(1, 2, 3, 4)
(2, 3, 4, 5)
(3, 4, 5, 6)
(4, 5, 6, 7)
(4, 5, 6, 8)
(5, 6, 8, 9)
(7, 6, 8, 9)


In [8]:
print(test1)

CCCCC=C=C(C)CC	



In [9]:
test1.make3D()
print(test1)

CCCCC=C=C(C)CC	



In [10]:
for torsion in openbabel.OBMolTorsionIter(test1.OBMol):
    print(torsion)

(10, 0, 1, 2)
(10, 0, 1, 13)
(10, 0, 1, 14)
(11, 0, 1, 2)
(11, 0, 1, 13)
(11, 0, 1, 14)
(12, 0, 1, 2)
(12, 0, 1, 13)
(12, 0, 1, 14)
(0, 1, 2, 3)
(0, 1, 2, 15)
(0, 1, 2, 16)
(13, 1, 2, 3)
(13, 1, 2, 15)
(13, 1, 2, 16)
(14, 1, 2, 3)
(14, 1, 2, 15)
(14, 1, 2, 16)
(1, 2, 3, 4)
(1, 2, 3, 17)
(1, 2, 3, 18)
(15, 2, 3, 4)
(15, 2, 3, 17)
(15, 2, 3, 18)
(16, 2, 3, 4)
(16, 2, 3, 17)
(16, 2, 3, 18)
(2, 3, 4, 5)
(2, 3, 4, 19)
(17, 3, 4, 5)
(17, 3, 4, 19)
(18, 3, 4, 5)
(18, 3, 4, 19)
(3, 4, 5, 6)
(19, 4, 5, 6)
(4, 5, 6, 7)
(4, 5, 6, 8)
(5, 6, 7, 20)
(5, 6, 7, 21)
(5, 6, 7, 22)
(8, 6, 7, 20)
(8, 6, 7, 21)
(8, 6, 7, 22)
(5, 6, 8, 9)
(5, 6, 8, 23)
(5, 6, 8, 24)
(7, 6, 8, 9)
(7, 6, 8, 23)
(7, 6, 8, 24)
(6, 8, 9, 25)
(6, 8, 9, 26)
(6, 8, 9, 27)
(23, 8, 9, 25)
(23, 8, 9, 26)
(23, 8, 9, 27)
(24, 8, 9, 25)
(24, 8, 9, 26)
(24, 8, 9, 27)


In [ ]:
num_atoms = 10
atoms = [3,5,3,2,7,5,3,12,11,100]

fours = []
for i in range(num_atoms):
    four = []
    try:
        fours.append(atoms[i:i+4])
    except IndexError:
        if num_atoms % 4 == 0:
            fours.append()
            fours.append(atoms[-3:]+atoms[0])
            fours.append(atoms[-2:]+atoms[:3])
            fours.append(atoms[-1]+atoms[:4])
        elif num_atoms % 4 == 3:
            fours.append(atoms[-3:]+atoms[0])
            fours.append(atoms[-2:]+atoms[:3])
            fours.append(atoms[-1]+atoms[:4])
        elif num_atoms % 4 == 2:
            fours.append(atoms[-2:]+atoms[:3])
            fours.append(atoms[-1]+atoms[:4])
        elif num_atoms % 4 == 1:
            fours.append(atoms[-1]+atoms[:4])
        
print(fours)



In [12]:
l = [1,2,3,4]
print(l.reverse())

None


In [5]:
import pybel
#mol = pybel.Molecule(OBMol=mol)

mol = pybel.readstring("smi", "CCC")
mol.make3D()
mol.draw(filename="hello.png", show=False)
# mol write produces seg fault when output format png is used
#mol.write(format="png",filename="test2.png", overwrite=True)
#mol.write(format="sdf",filename="test2.sdf", overwrite=True)


In [4]:
pybel.outformats

{'acesin': 'ACES input format',
 'adf': 'ADF cartesian input format',
 'alc': 'Alchemy format',
 'ascii': 'ASCII format',
 'bgf': 'MSI BGF format',
 'box': 'Dock 3.5 Box format',
 'bs': 'Ball and Stick format',
 'c3d1': 'Chem3D Cartesian 1 format',
 'c3d2': 'Chem3D Cartesian 2 format',
 'cac': 'CAChe MolStruct format',
 'caccrt': 'Cacao Cartesian format',
 'cache': 'CAChe MolStruct format',
 'cacint': 'Cacao Internal format',
 'can': 'Canonical SMILES format',
 'cdjson': 'ChemDoodle JSON',
 'cdxml': 'ChemDraw CDXML format',
 'cht': 'Chemtool format',
 'cif': 'Crystallographic Information File',
 'ck': 'ChemKin format',
 'cml': 'Chemical Markup Language',
 'cmlr': 'CML Reaction format',
 'com': 'Gaussian 98/03 Input',
 'confabreport': 'Confab report format',
 'CONFIG': 'DL-POLY CONFIG',
 'CONTCAR': 'VASP format',
 'CONTFF': 'MDFF format',
 'copy': 'Copy raw text',
 'crk2d': 'Chemical Resource Kit diagram(2D)',
 'crk3d': 'Chemical Resource Kit 3D format',
 'csr': 'Accelrys/MSI Quanta CSR

In [ ]:
import openbabel